In [1]:
#!/usr/bin/env python
# coding: utf-8
from __future__ import print_function
from collections import defaultdict
import pandas as pd
import time
import numpy as np
import pyspark
from utilities import *
from scipy.stats import pearsonr

In [ ]:
sc = pyspark.SparkContext()

In [ ]:
inh = 'ATV'
#vec_string = 'dist_vec'
vec_string = 'count_vec'

In [ ]:
sparkfilename = get_sparkfilename(inh,vec_string)

In [ ]:
header = 'path_id,inh_type,vec_type,root_mutation,path_length,path_type,fraction_above_inh_threshold,path_variance'.split(",")
header = header[3:]

In [ ]:
print(header)

In [ ]:
c = {n:i for i,n in enumerate(header)}

In [ ]:
data = sc.textFile(sparkfilename).map(lambda x:x.split(",")[3:])
print(data.count())

In [ ]:
corr = lambda x:pearsonr(x[0],x[1])

In [ ]:
type_frac_var = data.map(lambda x:(x[c['path_type']], x[c['fraction_above_inh_threshold']], x[c['path_variance']]))

In [ ]:
type_listfrac_listvar = type_frac_var.map(lambda x:(x[0],(float(x[1]),float(x[2])))).groupByKey().map(lambda x:(x[0],separate(list(x[1]))))

In [ ]:
#type_listfrac_listvar.take(1)

In [ ]:
viz_scatter = type_listfrac_listvar.collectAsMap()

In [ ]:
viz_scatter.keys()

In [ ]:
def draw_scatter(x,y,ptsize):
    plt.scatter(x,y,s=ptsize, color = 'k')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 10, 10
keys = ['gains','looses','spikes']

In [ ]:
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
max_var

In [ ]:
spikes_frac,spikes_var = viz_scatter[keys[-1]]
max_spikes_var = max(spikes_var)
try:
    gains_frac, gains_var = viz_scatter[keys[0]]
    max_gains_var = max(gains_var)
except:
    max_gains_var = 0 
    gains_frac, gains_var = [],[]
try:
    looses_frac, looses_var = viz_scatter[keys[1]]
    max_looses_var = max(looses_var)
except: 
    max_looses_var = 0
    looses_frac, looses_var = [],[]
max_var = max(max_gains_var,max_looses_var, max_spikes_var)
plt.ylim([0,max_var])

In [ ]:
draw_scatter(spikes_frac,spikes_var,1)
plt.ylim([0,max_var])
plt.xlabel("Fraction above threshold",fontsize=15)
plt.ylabel("Variance",fontsize=15)
plt.title(inh+" "+vec_string+" "+keys[-1],fontsize=20)
plt.show()

In [ ]:
print(get_figfile(inh,vec_string,keys[-1]).split("/")[-1])

In [ ]:
draw_scatter(gains_frac,gains_var,1)
plt.ylim([0,max_var])
plt.xlabel("Fraction above threshold",fontsize=15)
plt.ylabel("Variance",fontsize=15)
plt.title(inh+" "+vec_string+" "+keys[0],fontsize=20)

In [ ]:
print(get_figfile(inh,vec_string,keys[0]).split("/")[-1])

In [ ]:
draw_scatter(looses_frac,looses_var,1)
plt.ylim([0,max_var])
plt.xlabel("Fraction above threshold",fontsize=15)
plt.ylabel("Variance",fontsize=15)
plt.title(inh+" "+vec_string+" "+keys[1],fontsize=20)

In [ ]:
print(get_figfile(inh,vec_string,keys[1]).split("/")[-1])